# Amazon Review EDA

General EDA to understand the data, and create potential avenues for a model.

In [2]:
# Modules
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [4]:
# Load in data
reviews = pd.read_csv(Path(os.getcwd()).parents[0].joinpath("data", "amazon_reviews.csv"))

Took 18 seconds to load on OP computer. Let's see time on laptop.